In [1]:
import pickle

In [2]:
import pickle
file = open("y_real_msvdqa.pkl",'rb')
reals = pickle.load(file)

file = open("y_pred_msvdqa.pkl",'rb')
preds = pickle.load(file)
len(preds)

file = open("questions_actnet.pkl",'rb')
questions = pickle.load(file)
len(questions)

In [3]:
file = open("y_pred_msvdqa.pkl",'rb')
preds = pickle.load(file)
len(preds)

4901

In [4]:
file = open("questions_actnet.pkl",'rb')
questions = pickle.load(file)
len(questions)

7990

In [5]:
questions[4700]

'What is the identity of a brunette woman? Answer the question using a single word.'

In [6]:
preds[4700]

'New reporter'

In [7]:
reals[4700]

'Reporter'

In [8]:
reals[-5:]

['Colleague', 'Colleague', 'Opponent', 'Family', 'Opponent']

In [9]:
preds[-5:]

['They seem to be talking to each other',
 'None',
 'Biker that falls down',
 'They play together',
 'Friends']

In [10]:
import openai

openai.api_key = "EMPTY"
openai.base_url = "http://localhost:8000/v1/"

model = "vicuna-13b-v1.5"
prompt = "Once upon a time"

# create a completion
completion = openai.completions.create(model=model, prompt=prompt, max_tokens=64)
# print the completion
print(prompt + completion.choices[0].text)

# create a chat completion
completion = openai.chat.completions.create(
  model=model,
  messages=[{"role": "user", "content": "Hello! What is your name?"}]
)
# print the completion
print(completion.choices[0].message.content)

Once upon a time, in a kingdom far, far away, there was a beautiful princess named Sophia. She had long, flowing hair as golden as the sun, and eyes that sparkled like the morning dew on a rose petal.

Sophia lived in a grand castle, surrounded by lush gard
Hello! You can call me Vicuna, and I was trained by Large Model Systems Organization (LMSYS) researchers as a language model.


In [12]:
import ast

answers = []
scores = []
for ind, elem in enumerate(preds):
    question = questions[ind].split('?')[0]
    answer = reals[ind]
    pred = elem
    
    completion = openai.chat.completions.create(
                    model=model,
                    messages=[
                        {
                            "role": "system",
                            "content": 
                                "You are an intelligent chatbot designed for evaluating the correctness of generative outputs for question-answer pairs. "
                                "Your task is to compare the predicted answer with the correct answer and determine if they match meaningfully. Here's how you can accomplish the task:"
                                "------"
                                "##INSTRUCTIONS: "
                                "- Focus on the meaningful match between the predicted answer and the correct answer.\n"
                                "- Consider synonyms or paraphrases as valid matches.\n"
                                "- Evaluate the correctness of the prediction compared to the answer."
                        },
                        {
                            "role": "user",
                            "content":
                                "Please evaluate the following video-based question-answer pair:\n\n"
                                f"Question: {question}\n"
                                f"Correct Answer: {answer}\n"
                                f"Predicted Answer: {pred}\n\n"
                                "Provide your evaluation only as a yes/no and score, where the evaluation is 'yes' if the Predicted Answer and Correct Answer have the same meaning in the context of the Question, 'no' in other cases; the score is an integer value between 0 and 5, with 5 indicating the highest meaningful match. "
                                "Please generate the response in the form of a Python dictionary string with keys 'pred' and 'score', where value of 'pred' is  a string of 'yes' or 'no' and value of 'score' is in INTEGER, not STRING."
                                "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                                "For example, your response should look like this: {'pred': 'yes', 'score': 4.8}."
                        }
                    ]
    )
    # Convert response to a Python dictionary.
    if (ind%100 ==0):
        print (ind)
        print (print (question, answer, pred))
        print (ast.literal_eval(completion.choices[0].message.content)['pred'])
        print (ast.literal_eval(completion.choices[0].message.content)['score'])
    try:
        answers.append(ast.literal_eval(completion.choices[0].message.content)['pred'])
        scores.append(ast.literal_eval(completion.choices[0].message.content)['score'])
    except:
        answers.append(ast.literal_eval(completion.choices[0].message.content.split("\n\nExplanation")[0])['pred'])
        scores.append(ast.literal_eval(completion.choices[0].message.content.split("\n\nExplanation")[0])['score'])

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
0
Is the athlete wearing trousers No No
None
yes
5.0
100
Is the athlete wearing shorts Yes Yes
None
yes
5.0
200
How about the dance in the video Good looking The woman wearing a black dress is practicing ballet moves, making a cartwheel, and dancing with the birds on the tiles.
None
no
1.5
300
How many people are playing games in the video 4 4
None
yes
5
400
How many people are there in the video 2 1
None
no
2.5
500
How many people are there in the video 1 1
None
yes
5
600
How many women are there in the video 2 1
None
no
0
700
Is the man holding the rope outdoors Yes Yes
None
yes
5.0
800
Is the person in the video indoors Yes Yes
None
yes
5.0
900
Does the person in blue have long hair No No
None
yes
4.8
1000
Is the person in the video wearing a white clothes Yes Yes
None
yes
5
1100
Is the woman in the video wearing a white dress Yes 

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
import pickle
with open('vicunia15B_actnet.pkl', 'wb') as f:
        pickle.dump(answers, f)
with open('vicunia15B_actnet.pkl', 'wb') as f:
        pickle.dump(scores, f)

In [15]:
import numpy as np
np.mean(scores)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


3.5898774080560423

In [17]:
len(answers)

2855

In [16]:
yes_count = 0
no_count = 0

for elem in answers:
   
    # Computing accuracy
    if "yes" in elem.lower():
        yes_count += 1
    elif "no" in elem.lower():
        no_count += 1

accuracy = yes_count / (yes_count + no_count)
accuracy

0.45884413309982486